# Primary Text Processing

## Import

### Libraries

In [1]:
import os 
import codecs

from backend import *

### Definitions

In [2]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

In [3]:
texts = './texts/fiction/'

libCols = ['author','pub_year','title','text']
tokenOHCO = ['w_id','part_num','para_num', 'sent_num', 'token_num']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

## Primary Texts

### Library

In [4]:
libDf = pd.DataFrame(columns = libCols)
for t in os.listdir(texts): 
    if t[-4:] == '.txt': 
        #print(t)
        info = re.match(r'(\w+)-(\d{4})-(.+).txt', t)
        with codecs.open(texts+t, 'r', encoding='windows-1251') as f: 
            textytext = f.read()
        libDf = libDf.append({
            'author': info.group(1),
            'pub_year': int(info.group(2)), 
            'title': info.group(3), 
            'text': textytext
        }, ignore_index=True)
        
libDf = libDf.sort_values(libCols[1:3]).reset_index().drop(['index'], axis=1)
libDf.index.name = 'w_id'
textDf = libDf[[libCols[3]]]
libDf = libDf.drop(columns=[libCols[3]])
libDf

,author,pub_year,title
w_id,,,
0,gorkii,1900,troe
1,andreev,1903,zhizn-vasiliia-fiveiskogo
2,andreev,1904,gubernator
3,andreev,1905,k-zvezdam
4,andreev,1905,khristiane
5,andreev,1905,tak-bylo
6,gorkii,1906,mat
7,andreev,1906,savva-ignis-sanat
8,andreev,1907,iuda-iskariot


In [12]:
nat_parse(textDf.loc[:0])

,text
w_id,
0,Среди лесов Керженца рассеяно много оди...


In [ ]:
TestDf = nat_parse(textDf.iloc[:1]).set_index(['p_id', 'token_num'])

### Tokens

In [ ]:
tokenDf = textDf.text.str.split('\n\n\n', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:2]
tokenDf = tokenDf.rename(columns={0:'parts'})
tokenDf = tokenDf.parts.str.split('\n\s*\n', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:3]
tokenDf = tokenDf.rename(columns={0:'paras'})
tokenDf = tokenDf.paras.str.split(r'!|\?|\.|;', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:4]
#tokenDf 
sentDf = tokenDf = tokenDf.rename(columns={0:'sents'})
tokenDf = tokenDf.sents.str.split(r'\n|\s+', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO
tokenDf = tokenDf.rename(columns={0:'token'})
tokenDf = tokenDf.token.apply(lambda x: x.lower().strip(r"\W\|\]\}\[\{\.\'\"\?;:,<>/1234567890")).to_frame()
tokenDf = tokenDf[tokenDf['token'].str.contains(r'\w+')]
tokenDf

In [ ]:
tokenDf.token.value_counts().to_frame().reset_index().iloc[:60]

In [ ]:
sentDf.sents.apply(lambda x: x.strip(r"--")).to_frame()